import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=480
cls_num=2
btch=20
for i in range (0,cls_num):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=DistalPhalanxOutlineCorrect(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (0,cls_num):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (276, 81)
first_test_shape= (600, 81)
classes_quantity= 2
tr_lbls=	 {0, 1}
Count_labels= [161]
max(train_feature_Altitude)= 2.4602
min(train_feature_Altitude)= -2.1799
first_train_sample=
 [ 1.       -0.88425  -0.76511  -0.6264   -0.30459  -0.014302  0.29922
  0.51262   0.62572   0.89455   1.053     1.1322    1.2672    1.4321
  1.4845    1.2757    1.1589    1.0053    0.81736   0.50439   0.37885
  0.14581  -0.20947  -0.57872  -0.67481  -0.79999  -0.93504  -0.78816
 -0.57744  -0.27191   0.061242  0.41037   0.76072   1.1348    1.5046
  1.6696    1.6999    1.5531    1.3937    1.4704    1.5877    1.6902
  1.7647    1.5246    1.1461    0.76139   0.42848   0.076242 -0.2335
 -0.51671  -0.81834  -0.080823 -0.27898  -0.46208  -0.69433  -0.92273
 -1.1133   -1.3737   -1.5067   -1.5621   -1.3708   -1.102    -0.86298
 -0.53403  -0.3252   -0.1899   -0.3366   -0.45422  -0.60653  -0.78383
 -1.0497   -1.3447   -1.4977   -1.5734   -1.4749   -1.2418   -1.0149
 -0.78176  -0.61354  -0.306

In [3]:
np.shape(ecg)

(437, 481)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
0 115	1 161	
 max = 161


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (0,cls_num):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  0 train shape =  (115, 481)
Up to class  1 train shape =  (276, 481)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[115 161]


In [8]:
xtrain[:,-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (0,cls_num):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (0,cls_num):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 0  >> 
max magnitude class 0  =  2.4659367123545555
min magnitude class 0  =  -2.1616024973492727
after normalizing >>
max magnitude class 0  =  0.9764289207494858
min magnitude class 0  =  -0.9718534363802719

 cls 1  >> 
max magnitude class 1  =  2.095753113350402
min magnitude class 1  =  -1.994499999999999
after normalizing >>
max magnitude class 1  =  0.8205745579542003
min magnitude class 1  =  -0.9015000946440819
each class and its windows =
0 222	1 378	
 max instance in classes of test windows = 378


In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.9718534363802719
max tst = 0.9764289207494858


In [14]:
len(wndws_test)

600

In [15]:
np.shape(wndws_test)

(600, 481)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(161, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (276, 480)
y_train =>  (276,)
X_test  =>  (600, 480)
y_test  =>  (600,)
X_valid  =>  (276, 480)
y_valid  =>  (276,)


In [22]:
y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [23]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [24]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
4/4 [==============================] - 6s 224ms/step - loss: 9.3917 - accuracy: 0.5217 - val_loss: 1.0381 - val_accuracy: 0.4167
Epoch 2/1000
4/4 [==============================] - 0s 31ms/step - loss: 0.8577 - accuracy: 0.4529 - val_loss: 0.9942 - val_accuracy: 0.4819
Epoch 3/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.7013 - accuracy: 0.6159 - val_loss: 0.9983 - val_accuracy: 0.4167
Epoch 4/1000
4/4 [==============================] - 0s 30ms/step - loss: 0.6637 - accuracy: 0.6087 - val_loss: 0.9201 - val_accuracy: 0.5833
Epoch 5/1000
4/4 [==============================] - 0s 26ms/step - loss: 0.8306 - accuracy: 0.5109 - val_loss: 0.9301 - val_accuracy: 0.5833
Epoch 6/1000
4/4 [==============================] - 0s 27ms/step - loss: 0.6541 - accuracy: 0.6159 - val_loss: 0.8944 - val_accuracy: 0.5833
Epoch 7/1000
4/4 [==============================] - 0s 31ms/step - loss: 0.7319 - accuracy: 0.6196 - val_loss: 0.8782 - val_accuracy: 0.6304
Epoch 8/1000

Epoch 59/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.2545 - accuracy: 0.9058 - val_loss: 0.7694 - val_accuracy: 0.4239
Epoch 60/1000
4/4 [==============================] - 0s 27ms/step - loss: 0.3757 - accuracy: 0.8551 - val_loss: 0.6916 - val_accuracy: 0.6522
Epoch 61/1000
4/4 [==============================] - 0s 27ms/step - loss: 0.3059 - accuracy: 0.8768 - val_loss: 0.7513 - val_accuracy: 0.6087
Epoch 62/1000
4/4 [==============================] - 0s 31ms/step - loss: 0.3063 - accuracy: 0.8406 - val_loss: 0.6259 - val_accuracy: 0.6884
Epoch 63/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.4814 - accuracy: 0.7935 - val_loss: 0.7180 - val_accuracy: 0.5543
Epoch 64/1000
4/4 [==============================] - 0s 27ms/step - loss: 0.2432 - accuracy: 0.8841 - val_loss: 0.6953 - val_accuracy: 0.6341
Epoch 65/1000
4/4 [==============================] - 0s 27ms/step - loss: 0.2690 - accuracy: 0.8804 - val_loss: 0.6999 - val_accuracy: 0.6449
Epoch 

4/4 [==============================] - 0s 28ms/step - loss: 0.0720 - accuracy: 0.9565 - val_loss: 0.0459 - val_accuracy: 0.9819
Epoch 174/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.0451 - accuracy: 0.9819 - val_loss: 0.5176 - val_accuracy: 0.9312
Epoch 175/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.9538 - accuracy: 0.8841 - val_loss: 0.1975 - val_accuracy: 0.9348
Epoch 176/1000
4/4 [==============================] - 0s 27ms/step - loss: 0.1321 - accuracy: 0.9565 - val_loss: 0.0851 - val_accuracy: 0.9783
Epoch 177/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.0627 - accuracy: 0.9819 - val_loss: 0.0323 - val_accuracy: 0.9891
Epoch 178/1000
4/4 [==============================] - 0s 27ms/step - loss: 0.0096 - accuracy: 1.0000 - val_loss: 0.0397 - val_accuracy: 0.9819
Epoch 179/1000
4/4 [==============================] - 0s 30ms/step - loss: 0.1342 - accuracy: 0.9674 - val_loss: 0.0508 - val_accuracy: 0.9855
Epoch 180/1000

In [25]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [26]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

19/19 [==============================] - 0s 11ms/step
array([[155,  67],
       [ 78, 300]], dtype=int64)
              precision    recall  f1-score   support

           1       0.67      0.70      0.68       222
           2       0.82      0.79      0.81       378

    accuracy                           0.76       600
   macro avg       0.74      0.75      0.74       600
weighted avg       0.76      0.76      0.76       600



In [27]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
4/4 [==============================] - 2s 215ms/step - loss: 15.9872 - accuracy: 0.4565 - val_loss: 0.9904 - val_accuracy: 0.4167
Epoch 2/1000
4/4 [==============================] - 0s 37ms/step - loss: 1.1554 - accuracy: 0.4457 - val_loss: 1.0209 - val_accuracy: 0.4167
Epoch 3/1000
4/4 [==============================] - 0s 43ms/step - loss: 0.7420 - accuracy: 0.5217 - val_loss: 0.8815 - val_accuracy: 0.5833
Epoch 4/1000
4/4 [==============================] - 0s 36ms/step - loss: 0.9139 - accuracy: 0.5833 - val_loss: 0.9544 - val_accuracy: 0.5833
Epoch 5/1000
4/4 [==============================] - 0s 37ms/step - loss: 0.7401 - accuracy: 0.5399 - val_loss: 0.9169 - val_accuracy: 0.5833
Epoch 6/1000
4/4 [==============================] - 0s 37ms/step - loss: 0.7293 - accuracy: 0.6014 - val_loss: 0.8445 - val_accuracy: 0.5833
Epoch 7/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.6477 - accuracy: 0.6304 - val_loss: 0.7989 - val_accuracy: 0.5833
Epoch 8/100

Epoch 59/1000
4/4 [==============================] - 0s 36ms/step - loss: 0.3219 - accuracy: 0.8696 - val_loss: 0.6396 - val_accuracy: 0.6920
Epoch 60/1000
4/4 [==============================] - 0s 36ms/step - loss: 0.4634 - accuracy: 0.8297 - val_loss: 0.6845 - val_accuracy: 0.6268
Epoch 61/1000
4/4 [==============================] - 0s 37ms/step - loss: 0.4222 - accuracy: 0.8043 - val_loss: 0.6808 - val_accuracy: 0.6993
Epoch 62/1000
4/4 [==============================] - 0s 37ms/step - loss: 0.3103 - accuracy: 0.8732 - val_loss: 0.6685 - val_accuracy: 0.6703
Epoch 63/1000
4/4 [==============================] - 0s 37ms/step - loss: 0.2811 - accuracy: 0.8587 - val_loss: 0.7094 - val_accuracy: 0.4203
Epoch 64/1000
4/4 [==============================] - 0s 36ms/step - loss: 0.6608 - accuracy: 0.7645 - val_loss: 0.6828 - val_accuracy: 0.5254
Epoch 65/1000
4/4 [==============================] - 0s 37ms/step - loss: 0.3138 - accuracy: 0.8623 - val_loss: 0.6431 - val_accuracy: 0.7029
Epoch 

4/4 [==============================] - 0s 37ms/step - loss: 0.1744 - accuracy: 0.9638 - val_loss: 0.0955 - val_accuracy: 0.9710
Epoch 174/1000
4/4 [==============================] - 0s 37ms/step - loss: 0.0659 - accuracy: 0.9819 - val_loss: 0.1954 - val_accuracy: 0.9420
Epoch 175/1000
4/4 [==============================] - 0s 37ms/step - loss: 0.3953 - accuracy: 0.9384 - val_loss: 0.2712 - val_accuracy: 0.8623
Epoch 176/1000
4/4 [==============================] - 0s 36ms/step - loss: 0.2483 - accuracy: 0.9275 - val_loss: 0.1324 - val_accuracy: 0.9457
Epoch 177/1000
4/4 [==============================] - 0s 36ms/step - loss: 0.1690 - accuracy: 0.9384 - val_loss: 0.0462 - val_accuracy: 0.9855
Epoch 178/1000
4/4 [==============================] - 0s 43ms/step - loss: 0.0539 - accuracy: 0.9855 - val_loss: 0.0315 - val_accuracy: 0.9891
Epoch 179/1000
4/4 [==============================] - 0s 37ms/step - loss: 0.0952 - accuracy: 0.9674 - val_loss: 0.2634 - val_accuracy: 0.8913
Epoch 180/1000

In [28]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

19/19 [==============================] - 0s 10ms/step
array([[178,  44],
       [ 92, 286]], dtype=int64)
              precision    recall  f1-score   support

           1       0.66      0.80      0.72       222
           2       0.87      0.76      0.81       378

    accuracy                           0.77       600
   macro avg       0.76      0.78      0.77       600
weighted avg       0.79      0.77      0.78       600

